Write implementation on PyTorch for TransE model (you can use TorchGeometric or DGL library for working with graphs) and train your model on WordNet18RR dataset (you can use loaded dataset from any graph library).

As a result, you must provide a link to github (or gitlab) with all the source code.
The readability of the code, the presence of comments, type annotations, and the quality of the code as a whole will be taken into account when checking the test case.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch_geometric.datasets import WordNet18RR
from torch_geometric.loader import DataLoader
import pytorch_lightning as pl

l1_norm = lambda x, y: torch.linalg.norm((x-y), ord=1)
l2_norm = lambda x, y: torch.linalg.norm((x-y), ord=2)

c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\pkg_resources\__init__.py:122: PkgResourcesDeprecationWarning: otobuf is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
class TransE(pl.LightningModule):
    def __init__(self, n_entities, n_relations, margin=1, emb_dim=50, distance_func=l1_norm) -> None:
        super().__init__()
        self.margin = margin
        self.emb_dim = emb_dim
        self.distance_func = distance_func

        val = 6/np.sqrt(emb_dim)
        entity_size = (n_entities, emb_dim)
        relation_size = (n_relations, emb_dim)
        self.entity_mat = torch.from_numpy(np.random.uniform(low=-val, high=val, size=entity_size))
        self.relation_math = torch.from_numpy(np.random.uniform(low=val, high=val, size=relation_size))
    
    def training_step(self, batch, batch_idx):
        pass

In [3]:
dataset = WordNet18RR('./WordNet18RR/')
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [43]:
data = torch.load("./WordNet18RR/processed/data.pt")[0]
data

Data(edge_index=[2, 93003], edge_type=[93003], train_mask=[93003], val_mask=[93003], test_mask=[93003], num_nodes=40943)

In [49]:
model = TransE(n_entities=data.num_nodes,
               n_relations=data.num_edges)